In [1]:
%config InlineBackend.figure_format = 'retina'

import pandas
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

df = src_df = pandas.read_csv('libavif.csv')
df['size_kb'] = df['size'] / 1000
df['data'] = df.agg('<tt>{0[size_kb]:5.2f}&nbsp;<br>{0[perf]:5.2f}&nbsp;<br>{0[dssim]:7.3f}</tt>'.format, axis=1)

df

,fname,speed,quality,outname,size,enc_time,perf,dssim,size_kb,data
0,mars,5,10,mars.s5.q10,202185,2.870649,0.487271,0.048149,202.185,<tt>202.19&nbsp;<br> 0.49&nbsp;<br> 0.048</tt>
1,mars,6,10,mars.s6.q10,200660,0.556433,2.513840,0.053448,200.660,<tt>200.66&nbsp;<br> 2.51&nbsp;<br> 0.053</tt>
2,mars,7,10,mars.s7.q10,226479,0.381568,3.665882,0.044093,226.479,<tt>226.48&nbsp;<br> 3.67&nbsp;<br> 0.044</tt>
3,mars,8,10,mars.s8.q10,228385,0.215498,6.490938,0.045427,228.385,<tt>228.38&nbsp;<br> 6.49&nbsp;<br> 0.045</tt>
4,mars,9,10,mars.s9.q10,248481,0.123600,11.317044,0.045413,248.481,<tt>248.48&nbsp;<br>11.32&nbsp;<br> 0.045</tt>
...,...,...,...,...,...,...,...,...,...,...
535,parrots,5,44,parrots.s5.q44,7077,0.437404,0.898977,0.751434,7.077,<tt> 7.08&nbsp;<br> 0.90&nbsp;<br> 0.751</tt>
536,parrots,6,44,parrots.s6.q44,7145,0.132920,2.958301,0.822938,7.145,<tt> 7.14&nbsp;<br> 2.96&nbsp;<br> 0.823</tt>
537,parrots,7,44,parrots.s7.q44,8065,0.077549,5.070550,0.803646,8.065,<tt> 8.06&nbsp;<br> 5.07&nbsp;<br> 0.804</tt>
538,parrots,8,44,parrots.s8.q44,8172,0.045371,8.666671,0.843579,8.172,<tt> 8.17&nbsp;<br> 8.67&nbsp;<br> 0.844</tt>


In [2]:
display(HTML("""<table>
<tr><th>Each cell:
<tr><td>
Size in kb<br>
Megapixels/s<br>
DSSIM mectric
</table>
"""))

tables = []

for fname in src_df.fname.unique():
    df = src_df[src_df.fname == fname]
    df = df.pivot(index='quality', columns='speed', values='data')
    tables.append(f"<h1>{fname}</h1>" + df.to_html(escape=False))

display(HTML("".join(
    f'<div style="float:left">{table}</div>'
    for table in tables
)))

speed,5,6,7,8,9
quality,,,,,
10,202.19 0.49 0.048,200.66 2.51 0.053,226.48 3.67 0.044,228.38 6.49 0.045,248.48 11.32 0.045
12,172.39 0.52 0.059,173.75 2.90 0.066,192.44 3.51 0.056,192.51 6.53 0.058,211.46 11.20 0.058
14,150.74 0.54 0.071,154.09 2.99 0.079,167.03 4.14 0.069,166.23 6.55 0.071,184.83 10.93 0.072
16,132.92 0.61 0.084,137.32 2.99 0.093,148.16 3.90 0.084,147.38 7.04 0.087,165.88 11.67 0.087
18,119.94 0.61 0.097,124.52 3.01 0.108,130.52 4.20 0.101,130.84 7.23 0.105,147.85 11.75 0.105
20,108.08 0.65 0.112,112.91 3.06 0.124,116.48 4.23 0.119,118.53 7.61 0.122,135.08 12.22 0.124
22,99.15 0.67 0.126,102.92 2.40 0.138,104.87 4.70 0.139,107.38 6.30 0.142,123.49 12.11 0.144
24,90.45 0.67 0.144,93.82 2.41 0.158,95.94 4.70 0.159,98.14 7.47 0.164,114.07 13.17 0.165
26,78.38 0.72 0.177,81.27 2.29 0.193,87.51 4.56 0.183,89.55 6.19 0.189,104.72 13.16 0.191
